In [1]:
import ROOT as rt
import numpy as np
np.random.seed(1234)

sh: root-config: command not found
Error in <TMacOSXSystem::GetFromPipe>: command "root-config --has-dataframe" returned 32512


Welcome to JupyROOT 6.28/00


# Just-in-time compilation of C++ function in PyROOT

In [2]:
rt.gInterpreter.Declare('''
float largest_sum(float* v1, float* v2, std::size_t size){
    float r = -999.f;
    for (size_t il = 0; il < size; il++) {
        for (size_t i2 = 0; i2 < size; i2++) {
            const auto tmp = v1[il] + v2[2];
            if (tmp > r) r = tmp;
        }
    }
    return r;
}
''');

In [3]:
size = 100
v1 = np.random.rand(size).astype(np.float32)
v2 = np.random.rand(size).astype(np.float32)

In [4]:
%%timeit

rt.largest_sum(v1, v2, size)

43.7 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
def largest_sum(x1, x2):
    r = -999.0
    for el in x1:
        for e2 in x2:
            tmp = el + e2
            if tmp > r: r = tmp
    return r

In [6]:
%%timeit

largest_sum(v1, v2)

686 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%%bash
cat analysis.hxx

cat analysis.cxx

g++ -Ofast -shared -o libanalysis.so analysis.cxx

#include <cstddef>
float optimized_largest_sum(float* v1, float* v2, std::size_t size);

# include "analysis.hxx"
float optimized_largest_sum(float* v1, float* v2, std::size_t size){
  float r = -999.f;
  for(size_t i1 = 0; i1 < size; i1++){
    for(size_t i2 = 0; i2 < size; i2++){
      float tmp = v1[i1] + v2[i2];
      if(tmp > r) r = tmp;
    }
  }
  return r;
}


In [8]:
rt.gInterpreter.Declare('#include "analysis.hxx"');
rt.gSystem.Load('libanalysis.so');

In [9]:
%%timeit
rt.optimized_largest_sum(v1, v2, size)

1.96 µs ± 3.33 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
print('PyROOT: ', rt.largest_sum(v1, v2, size))
print('Native Python: ', largest_sum(v1, v2) )
print('PyROOT (optimized):', rt.optimized_largest_sum(v1, v2, size) )

PyROOT:  1.788948655128479
Native Python:  1.9867353
PyROOT (optimized): 1.9867353439331055
